In [49]:
from __future__ import absolute_import, division, print_function, unicode_literals
# from keras import backend as K
import tensorflow as tf
import keras
import numpy as np
import tensorflow.contrib.eager as tfe
from keras.preprocessing import sequence
from sklearn.model_selection import train_test_split
import time



print(tf.__version__)

1.14.0


In [8]:
tfe.enable_eager_execution(device_policy=tfe.DEVICE_PLACEMENT_SILENT)

In [496]:
imdb = keras.datasets.imdb
num_features = 20000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=num_features)
print("Training entries: {}, labels: {}".format(len(train_data), len(train_labels)))

Training entries: 25000, labels: 25000


In [304]:
# A dictionary mapping words to an integer index
word_index = imdb.get_word_index()

# The first indices are reserved
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

In [305]:
print(len(word_index))

88588


In [497]:
maxlen = 80
x_train = sequence.pad_sequences(train_data, maxlen=maxlen,padding='post')
x_test = sequence.pad_sequences(test_data, maxlen=maxlen,padding='post')
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

x_train shape: (25000, 80)
x_test shape: (25000, 80)


In [307]:
print(train_labels)

[1 0 0 ... 0 1 0]


In [225]:
# train_labels = tf.one_hot(train_labels, depth=2)
# test_labels = tf.one_hot(test_labels,depth=2)

InternalError: Could not find valid device for node.
Node: {{node OneHot}}
All kernels registered for op OneHot :
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT64]
  device='CPU'; TI in [DT_INT32]; T in [DT_INT64]
  device='CPU'; TI in [DT_INT64]; T in [DT_INT64]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT32]
  device='CPU'; TI in [DT_INT32]; T in [DT_INT32]
  device='CPU'; TI in [DT_INT64]; T in [DT_INT32]
  device='CPU'; TI in [DT_UINT8]; T in [DT_UINT16]
  device='CPU'; TI in [DT_INT32]; T in [DT_UINT16]
  device='CPU'; TI in [DT_INT64]; T in [DT_UINT16]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT16]
  device='CPU'; TI in [DT_INT32]; T in [DT_INT16]
  device='CPU'; TI in [DT_INT64]; T in [DT_INT16]
  device='CPU'; TI in [DT_UINT8]; T in [DT_UINT8]
  device='CPU'; TI in [DT_INT32]; T in [DT_UINT8]
  device='CPU'; TI in [DT_INT64]; T in [DT_UINT8]
  device='CPU'; TI in [DT_UINT8]; T in [DT_INT8]
  device='CPU'; TI in [DT_INT32]; T in [DT_INT8]
  device='CPU'; TI in [DT_INT64]; T

In [308]:
decode_review(x_train[0])

"that played the <UNK> of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all"

In [593]:
BUFFER_SIZE = len(x_train)
BATCH_SIZE = 64
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dims = 32
lstm_units = 100

dataset = tf.data.Dataset.from_tensor_slices((x_train, train_labels)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [374]:
class Classifier(tf.keras.Model):
    def __init__(self,vocab_size,embedding_dim,lstm_units):
        super(Classifier, self).__init__()
        
        self.lstm_units = lstm_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(lstm_units)
        self.dense = tf.keras.layers.Dense(32,activation=tf.nn.relu)
        self.pred = tf.keras.layers.Dense(1,activation=None)
        

    def call(self, x,is_training):
        x = self.embedding(x)
#         num_samples = tf.shape(x)[0]
#         hidden = tf.zeros((BATCH_SIZE, self.lstm_units))
#         print(self.lstm.get_initial_state(x))
        print(x)
        output = self.lstm(x)     
#         print(output.shape)
#         o = tf.layers.dropout(output, rate=0.2, training=is_training)
#         o = self.dense(output)
        o = self.pred(o)
#         print(o)
#         o = tf.nn.softmax(o)
#         print(result)
        return o


In [319]:
model = Classifier(num_features, embedding_dims, lstm_units)

In [357]:
optimizer = tf.train.AdamOptimizer()


# [batch_size,class_size]
# [barch_size,]
def loss_function(real, pred):
#     print("real",real,"pred", pred)
#     weights = np.ones(real.shape[0])
#     print(weights)
    pred = tf.reshape(pred, [-1])
    loss_ = tf.losses.sigmoid_cross_entropy(real, pred) 
    print(loss_)
    return loss_

In [321]:
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(dataset):
        loss = 0
        with tf.GradientTape() as tape:
            output = model(inp,True)
#             print(output)
    
            loss = loss_function(targ, output)
                
        variables = model.trainable_variables
        gradients = tape.gradient(loss, variables)
        optimizer.apply_gradients(zip(gradients, variables))
        
        if batch % 100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         loss))
    # saving (checkpoint) the model every 2 epochs
#     if (epoch + 1) % 2 == 0:
#         checkpoint.save(file_prefix = checkpoint_prefix)
    
#     print('Epoch {} Loss {:.4f}'.format(epoch + 1,
#                                         total_loss / N_BATCH))
#     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

tf.Tensor(0.69208455, shape=(), dtype=float32)
Epoch 1 Batch 0 Loss 0.6921
tf.Tensor(0.69378436, shape=(), dtype=float32)
tf.Tensor(0.69377184, shape=(), dtype=float32)
tf.Tensor(0.69485843, shape=(), dtype=float32)
tf.Tensor(0.6943238, shape=(), dtype=float32)
tf.Tensor(0.69497895, shape=(), dtype=float32)
tf.Tensor(0.6947397, shape=(), dtype=float32)
tf.Tensor(0.6921652, shape=(), dtype=float32)
tf.Tensor(0.692135, shape=(), dtype=float32)
tf.Tensor(0.6938924, shape=(), dtype=float32)
tf.Tensor(0.69466925, shape=(), dtype=float32)
tf.Tensor(0.6926493, shape=(), dtype=float32)
tf.Tensor(0.69155556, shape=(), dtype=float32)
tf.Tensor(0.69358736, shape=(), dtype=float32)
tf.Tensor(0.69378465, shape=(), dtype=float32)
tf.Tensor(0.6910838, shape=(), dtype=float32)
tf.Tensor(0.6870157, shape=(), dtype=float32)
tf.Tensor(0.6896082, shape=(), dtype=float32)
tf.Tensor(0.694764, shape=(), dtype=float32)
tf.Tensor(0.69239664, shape=(), dtype=float32)
tf.Tensor(0.68541604, shape=(), dtype=float3

tf.Tensor(0.31136343, shape=(), dtype=float32)
tf.Tensor(0.42200258, shape=(), dtype=float32)
tf.Tensor(0.36288577, shape=(), dtype=float32)
tf.Tensor(0.56358826, shape=(), dtype=float32)
tf.Tensor(0.33568498, shape=(), dtype=float32)
tf.Tensor(0.36054122, shape=(), dtype=float32)
tf.Tensor(0.34853786, shape=(), dtype=float32)
tf.Tensor(0.4020235, shape=(), dtype=float32)
tf.Tensor(0.5255165, shape=(), dtype=float32)
tf.Tensor(0.7017703, shape=(), dtype=float32)
tf.Tensor(0.5687862, shape=(), dtype=float32)
tf.Tensor(0.48270518, shape=(), dtype=float32)
tf.Tensor(0.51063967, shape=(), dtype=float32)
tf.Tensor(0.37538433, shape=(), dtype=float32)
tf.Tensor(0.3813012, shape=(), dtype=float32)
tf.Tensor(0.5101967, shape=(), dtype=float32)
tf.Tensor(0.3833111, shape=(), dtype=float32)
tf.Tensor(0.46926093, shape=(), dtype=float32)
tf.Tensor(0.56017375, shape=(), dtype=float32)
tf.Tensor(0.48513597, shape=(), dtype=float32)
tf.Tensor(0.5348676, shape=(), dtype=float32)
tf.Tensor(0.43908137,

tf.Tensor(0.29213634, shape=(), dtype=float32)
tf.Tensor(0.43122754, shape=(), dtype=float32)
tf.Tensor(0.6011448, shape=(), dtype=float32)
tf.Tensor(0.3909574, shape=(), dtype=float32)
tf.Tensor(0.35181585, shape=(), dtype=float32)
tf.Tensor(0.50253356, shape=(), dtype=float32)
tf.Tensor(0.52070487, shape=(), dtype=float32)
tf.Tensor(0.4512058, shape=(), dtype=float32)
tf.Tensor(0.6128309, shape=(), dtype=float32)
tf.Tensor(0.4731127, shape=(), dtype=float32)
tf.Tensor(0.46964753, shape=(), dtype=float32)
tf.Tensor(0.47291926, shape=(), dtype=float32)
tf.Tensor(0.4058097, shape=(), dtype=float32)
tf.Tensor(0.4884255, shape=(), dtype=float32)
tf.Tensor(0.48621908, shape=(), dtype=float32)
tf.Tensor(0.5322679, shape=(), dtype=float32)
tf.Tensor(0.4676673, shape=(), dtype=float32)
tf.Tensor(0.45568553, shape=(), dtype=float32)
tf.Tensor(0.35048842, shape=(), dtype=float32)
tf.Tensor(0.4301595, shape=(), dtype=float32)
tf.Tensor(0.39329317, shape=(), dtype=float32)
tf.Tensor(0.35917976, s

tf.Tensor(0.31597602, shape=(), dtype=float32)
tf.Tensor(0.3474396, shape=(), dtype=float32)
tf.Tensor(0.44858965, shape=(), dtype=float32)
tf.Tensor(0.5278851, shape=(), dtype=float32)
tf.Tensor(0.38807186, shape=(), dtype=float32)
tf.Tensor(0.31569034, shape=(), dtype=float32)
tf.Tensor(0.3875463, shape=(), dtype=float32)
tf.Tensor(0.35177663, shape=(), dtype=float32)
tf.Tensor(0.5307752, shape=(), dtype=float32)
tf.Tensor(0.3101678, shape=(), dtype=float32)
tf.Tensor(0.39181668, shape=(), dtype=float32)
tf.Tensor(0.4011181, shape=(), dtype=float32)
tf.Tensor(0.40219802, shape=(), dtype=float32)
tf.Tensor(0.31463945, shape=(), dtype=float32)
tf.Tensor(0.42951497, shape=(), dtype=float32)
tf.Tensor(0.3665148, shape=(), dtype=float32)
tf.Tensor(0.33718765, shape=(), dtype=float32)
tf.Tensor(0.40518624, shape=(), dtype=float32)
tf.Tensor(0.44120365, shape=(), dtype=float32)
tf.Tensor(0.5325699, shape=(), dtype=float32)
tf.Tensor(0.38325864, shape=(), dtype=float32)
tf.Tensor(0.28351572,

Epoch 1 Batch 700 Loss 0.4485
tf.Tensor(0.24810493, shape=(), dtype=float32)
tf.Tensor(0.23754756, shape=(), dtype=float32)
tf.Tensor(0.27728885, shape=(), dtype=float32)
tf.Tensor(0.58648366, shape=(), dtype=float32)
tf.Tensor(0.4476722, shape=(), dtype=float32)
tf.Tensor(0.6213335, shape=(), dtype=float32)
tf.Tensor(0.40199092, shape=(), dtype=float32)
tf.Tensor(0.40212506, shape=(), dtype=float32)
tf.Tensor(0.22231562, shape=(), dtype=float32)
tf.Tensor(0.34540218, shape=(), dtype=float32)
tf.Tensor(0.3356986, shape=(), dtype=float32)
tf.Tensor(0.43459743, shape=(), dtype=float32)
tf.Tensor(0.41948974, shape=(), dtype=float32)
tf.Tensor(0.35847622, shape=(), dtype=float32)
tf.Tensor(0.18248996, shape=(), dtype=float32)
tf.Tensor(0.25813788, shape=(), dtype=float32)
tf.Tensor(0.37338865, shape=(), dtype=float32)
tf.Tensor(0.6362028, shape=(), dtype=float32)
tf.Tensor(0.28776434, shape=(), dtype=float32)
tf.Tensor(0.42673928, shape=(), dtype=float32)
tf.Tensor(0.17490932, shape=(), dt

...

tf.Tensor(0.0024219183, shape=(), dtype=float32)
tf.Tensor(0.0013141433, shape=(), dtype=float32)
tf.Tensor(0.15010032, shape=(), dtype=float32)
tf.Tensor(0.0056242384, shape=(), dtype=float32)
tf.Tensor(0.009286435, shape=(), dtype=float32)
tf.Tensor(0.001563426, shape=(), dtype=float32)
tf.Tensor(0.0015229598, shape=(), dtype=float32)
tf.Tensor(0.004256033, shape=(), dtype=float32)
tf.Tensor(0.0029445465, shape=(), dtype=float32)
tf.Tensor(0.0020712717, shape=(), dtype=float32)
tf.Tensor(0.009907473, shape=(), dtype=float32)
tf.Tensor(0.0010298878, shape=(), dtype=float32)
tf.Tensor(0.004381718, shape=(), dtype=float32)
tf.Tensor(0.007866287, shape=(), dtype=float32)
tf.Tensor(0.0026063256, shape=(), dtype=float32)
tf.Tensor(0.008807429, shape=(), dtype=float32)
tf.Tensor(0.04246759, shape=(), dtype=float32)
tf.Tensor(0.0040591955, shape=(), dtype=float32)
tf.Tensor(0.023984859, shape=(), dtype=float32)
tf.Tensor(0.002392767, shape=(), dtype=float32)
tf.Tensor(0.014896039, shape=(), 

tf.Tensor(0.0021837428, shape=(), dtype=float32)
tf.Tensor(0.008955488, shape=(), dtype=float32)
tf.Tensor(0.013665581, shape=(), dtype=float32)
tf.Tensor(0.0019148411, shape=(), dtype=float32)
tf.Tensor(0.002368385, shape=(), dtype=float32)
tf.Tensor(0.01385118, shape=(), dtype=float32)
tf.Tensor(0.013592728, shape=(), dtype=float32)
tf.Tensor(0.0033416024, shape=(), dtype=float32)
tf.Tensor(0.0012659454, shape=(), dtype=float32)
tf.Tensor(0.0044818935, shape=(), dtype=float32)
tf.Tensor(0.008192808, shape=(), dtype=float32)
tf.Tensor(0.0038604997, shape=(), dtype=float32)
Epoch 10 Batch 200 Loss 0.0039
tf.Tensor(0.0055063926, shape=(), dtype=float32)
tf.Tensor(0.0016994728, shape=(), dtype=float32)
tf.Tensor(0.0021223938, shape=(), dtype=float32)
tf.Tensor(0.005739715, shape=(), dtype=float32)
tf.Tensor(0.010367423, shape=(), dtype=float32)
tf.Tensor(0.003051255, shape=(), dtype=float32)
tf.Tensor(0.023490656, shape=(), dtype=float32)
tf.Tensor(0.14945258, shape=(), dtype=float32)
tf

tf.Tensor(0.0025168508, shape=(), dtype=float32)
tf.Tensor(0.013108415, shape=(), dtype=float32)
tf.Tensor(0.0052060643, shape=(), dtype=float32)
tf.Tensor(0.001161123, shape=(), dtype=float32)
tf.Tensor(0.042550333, shape=(), dtype=float32)
tf.Tensor(0.0023177094, shape=(), dtype=float32)
tf.Tensor(0.010037057, shape=(), dtype=float32)
tf.Tensor(0.0019755024, shape=(), dtype=float32)
tf.Tensor(0.002179096, shape=(), dtype=float32)
tf.Tensor(0.0059894766, shape=(), dtype=float32)
tf.Tensor(0.003475106, shape=(), dtype=float32)
tf.Tensor(0.0016494476, shape=(), dtype=float32)
tf.Tensor(0.0019005176, shape=(), dtype=float32)
tf.Tensor(0.012952636, shape=(), dtype=float32)
tf.Tensor(0.056715768, shape=(), dtype=float32)
tf.Tensor(0.0035054644, shape=(), dtype=float32)
tf.Tensor(0.005556019, shape=(), dtype=float32)
tf.Tensor(0.0026009209, shape=(), dtype=float32)
tf.Tensor(0.004912151, shape=(), dtype=float32)
tf.Tensor(0.045934267, shape=(), dtype=float32)
tf.Tensor(0.01812544, shape=(),

tf.Tensor(0.09353264, shape=(), dtype=float32)
tf.Tensor(0.0024587985, shape=(), dtype=float32)
tf.Tensor(0.004478124, shape=(), dtype=float32)
tf.Tensor(0.009362485, shape=(), dtype=float32)
tf.Tensor(0.003970296, shape=(), dtype=float32)
tf.Tensor(0.0016739332, shape=(), dtype=float32)
tf.Tensor(0.0017312651, shape=(), dtype=float32)
tf.Tensor(0.073935896, shape=(), dtype=float32)
tf.Tensor(0.0025475738, shape=(), dtype=float32)
tf.Tensor(0.20177366, shape=(), dtype=float32)
tf.Tensor(0.023431301, shape=(), dtype=float32)
tf.Tensor(0.012899089, shape=(), dtype=float32)
tf.Tensor(0.041159675, shape=(), dtype=float32)
tf.Tensor(0.0041865525, shape=(), dtype=float32)
tf.Tensor(0.0009986816, shape=(), dtype=float32)
tf.Tensor(0.0043596043, shape=(), dtype=float32)
tf.Tensor(0.082777545, shape=(), dtype=float32)
tf.Tensor(0.013781028, shape=(), dtype=float32)
tf.Tensor(0.079818696, shape=(), dtype=float32)
tf.Tensor(0.0016320908, shape=(), dtype=float32)
tf.Tensor(0.03779472, shape=(), dt

tf.Tensor(0.012850666, shape=(), dtype=float32)
tf.Tensor(0.021700298, shape=(), dtype=float32)
tf.Tensor(0.03942162, shape=(), dtype=float32)
tf.Tensor(0.014293159, shape=(), dtype=float32)
Epoch 10 Batch 700 Loss 0.0143
tf.Tensor(0.004729582, shape=(), dtype=float32)
tf.Tensor(0.012443587, shape=(), dtype=float32)
tf.Tensor(0.0073278146, shape=(), dtype=float32)
tf.Tensor(0.009089518, shape=(), dtype=float32)
tf.Tensor(0.0052023903, shape=(), dtype=float32)
tf.Tensor(0.023337357, shape=(), dtype=float32)
tf.Tensor(0.10387726, shape=(), dtype=float32)
tf.Tensor(0.0035373606, shape=(), dtype=float32)
tf.Tensor(0.021682236, shape=(), dtype=float32)
tf.Tensor(0.0039108493, shape=(), dtype=float32)
tf.Tensor(0.006621167, shape=(), dtype=float32)
tf.Tensor(0.0066683395, shape=(), dtype=float32)
tf.Tensor(0.008118032, shape=(), dtype=float32)
tf.Tensor(0.0034161666, shape=(), dtype=float32)
tf.Tensor(0.013268166, shape=(), dtype=float32)
tf.Tensor(0.0044962214, shape=(), dtype=float32)
tf.T

In [385]:
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, test_labels)).shuffle(BUFFER_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE, drop_remainder=True)
acc = []
attack_success = 0
total = 0
avg_change = 0
for (batch, (inp, targ)) in enumerate(test_dataset):
    out = model(inp,False)
    right = 0
    for idx,each in enumerate(out):
        real = targ[idx]
#         print(inp[idx])
#         print("actual label:",np.argmax(real),"pred label:",np.argmax(each))
        pred = 0
        real = real.numpy()
        if tf.sigmoid(each).numpy()[0] < 0.5:
            pred = 0
        else:
            pred = 1
        if pred == real:
            right += 1
#         if np.argmax(each) == np.argmax(real):
#             right += 1
    batch_acc = right/BATCH_SIZE
    acc.append(batch_acc)
    if batch == 100:
        break
print(np.mean(acc))

0.806930693069307


In [373]:
model.trainable_variables[0]

<tf.Variable 'classifier_45/embedding_45/embeddings:0' shape=(20000, 128) dtype=float32, numpy=
array([[-0.02718106, -0.02219358, -0.01925939, ...,  0.04519816,
        -0.02142727, -0.01421945],
       [ 0.07914771, -0.01114839, -0.04251073, ...,  0.06135698,
         0.08567193,  0.10764367],
       [-0.01019515, -0.01310153, -0.00912648, ...,  0.01744314,
        -0.01187728, -0.00231035],
       ...,
       [-0.04394948,  0.01303836, -0.00209283, ..., -0.00966215,
        -0.04624779,  0.08987358],
       [ 0.07939446, -0.07848275, -0.06281833, ...,  0.07691382,
         0.08478038,  0.12548651],
       [ 0.16306196, -0.07046987,  0.03912742, ...,  0.06019569,
         0.07130094,  0.16731149]], dtype=float32)>

In [367]:
model.layers

In [324]:
model.variables

[<tf.Variable 'classifier_45/embedding_45/embeddings:0' shape=(20000, 128) dtype=float32, numpy=
 array([[-0.02718106, -0.02219358, -0.01925939, ...,  0.04519816,
         -0.02142727, -0.01421945],
        [ 0.07914771, -0.01114839, -0.04251073, ...,  0.06135698,
          0.08567193,  0.10764367],
        [-0.01019515, -0.01310153, -0.00912648, ...,  0.01744314,
         -0.01187728, -0.00231035],
        ...,
        [-0.04394948,  0.01303836, -0.00209283, ..., -0.00966215,
         -0.04624779,  0.08987358],
        [ 0.07939446, -0.07848275, -0.06281833, ...,  0.07691382,
          0.08478038,  0.12548651],
        [ 0.16306196, -0.07046987,  0.03912742, ...,  0.06019569,
          0.07130094,  0.16731149]], dtype=float32)>,
 <tf.Variable 'classifier_45/lstm_44/kernel:0' shape=(128, 512) dtype=float32, numpy=
 array([[ 0.00261671,  0.11188858, -0.05251358, ...,  0.01174942,
         -0.0578456 ,  0.06984546],
        [-0.1174419 , -0.11761596,  0.04090131, ...,  0.10068762,
      

In [591]:
def hotflip_attack(x,gradients):
    embedding = model.embedding
    x_embedding = embedding(x)
    x_grad = gradients.values
    embedding_matrix = embedding.weights[0]
    # grad[x_t]^T * (x_t - x_p) is maximized 
    # The "Hotflip" attack described clearly in https://arxiv.org/abs/1903.06620, 
    # their code I used is here https://github.com/pmichel31415/translate/blob/paul/pytorch_translate/research/adversarial/adversaries/brute_force_adversary.py
    # embedding = [20000,128]  ~ to get x_t
    # x_embedding = [1,80,128] ~ x_t
    # x_grad = [80,128]        ~ grad[x_t]
    # 1 use einsum to compute the grad[x_t]^T * x_p. 
    x_grad = tf.reshape(x_grad,[1,x_grad.shape[0],x_grad.shape[1]])
    product1 = tf.einsum("bij,kj->bik", x_grad, embedding_matrix)
    print(product1.shape)
    # 2 compute grad[x_t]^T * x_t
    product2 = tf.einsum("bij,bij->bi", x_grad, x_embedding)
#     print(product2.shape)
    product2 = tf.expand_dims(product2,-1)
    print(product2.shape)
    # 3 we want the maximum of this difference to increase the loss
    neg_dir_dot_grad = -1*(product2 - product1)
    print(neg_dir_dot_grad.shape)
#     print(product2[0])
#     print("first column",product1[0,:,0])
#     print(neg_dir_dot_grad[0,:,0])
#     print("second column",product1[0,:,1])
#     print(neg_dir_dot_grad[0,:,1])
    
    # 4 maybe normalize it
    # 5 pick the max
    best = tf.argmax(neg_dir_dot_grad,2)
    # Now I have a tensor of size [1,80]. This means for each word i
    # I have a best perturbation generated from the vocab.
    # 6 this step chooses which particular word to attack. The heuristic
    # used here is to simply choose the word with max gradient norm.
    num_of_words = gradients.values.shape[0]
    grads = gradients.values
    grads_norm = tf.einsum("ij,ij->i",grads,grads)
    print(grads_norm)
    which_token = np.argmax(grads_norm)
    print("perturbed index:",which_token)
    print("perturbed token index:",best.numpy()[0][which_token])
    x_p = best.numpy()[0][which_token]
    new_x = np.array(x)[0]
    new_x[which_token] = x_p
    return best,new_x


def get_pred(model,x,y):
    output = model(x, False)
    pred = 0
    conf = tf.sigmoid(output).numpy()[0]
    if conf[0] < 0.5:
        pred = 0
    else:
        pred = 1
    
    print("\nreal:",new_y,"pred:",pred, "conf", conf[0],"\n")
    return output,y==pred
    

In [594]:
x = np.array(x_test[9])
y = np.array(test_labels[9])
new_x = np.array(x)
new_x = new_x.reshape((1,80))
new_y = np.array(y)
print(x_test.shape)
print("Initial sentence:")
print(decode_review(x))
x =x.reshape((1,80))
ay = tf.reshape(y, [-1])
with tf.GradientTape() as tape:
    output,same = get_pred(model,x,y)
count = 0
if same:
    while same:
        print("attack # %d ---------------------------- "%(count))
        count +=1
        with tf.GradientTape() as tape:
            x =x.reshape((1,80))
            output,same = get_pred(model,x,y)
            loss = loss_function(ay, output)
            variables = model.trainable_variables
            gradients = tape.gradient(loss, [variables[0]])
            candidates,x2 = hotflip_attack(x, gradients[0])
        print("perturbed sentences:")
        print(decode_review(x2))
        x = x2
        print(decode_review(candidates.numpy()[0]))
#         print(candidates.numpy()[0])
    else:
        print("attack succeed!")
else:
    print("wrong prediction!")


(25000, 80)
Initial sentence:
only weakness i can really point out is a certain to the script in some places which i think is due mostly to the way this film is a four cornered fight there simply isn't enough time to properly explore what's going on br br nevertheless this is a damn good film i highly recommend watching this in <UNK> with the first and then weeping for how good the series could have been had it continued under burton and keaton

real: 1 pred: 1 conf 0.99946016 

attack # 0 ---------------------------- 

real: 1 pred: 1 conf 0.99946016 

tf.Tensor(0.0005399682, shape=(), dtype=float32)
(1, 80, 20000)
(1, 80, 1)
(1, 80, 20000)
tf.Tensor(
[2.77862444e-09 1.71094983e-09 2.82698487e-09 1.43690038e-09
 3.93899402e-09 3.89581434e-09 4.73326223e-09 4.36873915e-09
 4.91033436e-09 1.23122526e-08 8.02283040e-09 6.42255404e-09
 2.94761016e-08 2.85613488e-08 2.80735222e-08 1.96402823e-08
 1.47411505e-08 1.59052096e-08 1.03039177e-08 7.38056638e-09
 1.56391131e-08 2.76398708e-08 3.2

In [585]:
new_x[0][79] = 6836
print(new_x)
print(decode_review(new_x[0]))

[[    4     2    14   720     9   617  3727    39   998  2445  1346  3862
     46    51   242    16    64     6   677   333  1494    10    10     2
      9     6    22    15     4  4198  1103 13002    12    18   148   460
   2534     5    12    18     4     2    12     2     7   968     5     2
     19     4  4271 18885     7     4 12603     2  7936    94   833  1488
     47    77    23    22  1537    37    28  4902     5    64  1076  3827
     23  3362  5161    11  5421   450  5122  6836]]
the <UNK> this sequence is shown repeatedly from various angles thus drawing out what probably was only a five second event br br <UNK> is a film that the revolutionary spirit celebrates it for those already committed and it for the <UNK> it <UNK> of fire and <UNK> with the senseless injustices of the decadent <UNK> regime its greatest impact has been on film students who have borrowed and only slightly improved on techniques invented in russia several generations tolerable


In [587]:
output = model(new_x, False)
pred = 0
conf = tf.sigmoid(output).numpy()[0][0]
if tf.sigmoid(output).numpy()[0] < 0.5:
    pred = 0
else:
    pred = 1
print("\nreal:",new_y,"pred:",pred, "conf", conf,"\n")


real: 1 pred: 0 conf 0.00036700023 

